In [2]:
host = '192.168.1.144'
port = 830
username = 'netconf'
password = 'abc123'

In [3]:
import ncclient
from ncclient import manager

with manager.connect_ssh(host=host,
                         port=port,
                         username=username,
                         password=password,
                         hostkey_verify=False,
                         timeout=120,
                         device_params={'name':'junos'}) as mgr:
    config: ncclient.xml_.NCElement = mgr.get_config(source='running')

In [4]:
from typing import List
from lxml.etree import _Element

users: List[_Element] = config.xpath('//configuration/system/login/user')
for user in users:
    print('-'*50)  # print separator
    label_prefixes = []  # collect parent element tag names
    for child in user.iter():  # iterates in document order
        if child.text is None:  # element has no text; children are other elements
            label_prefixes.append(child.tag)
        else:
            key = f"{'.'.join(label_prefixes)}.{child.tag}"
            value = child.text
            print(f"{key} : {value}")


--------------------------------------------------
user.name : netconf
user.uid : 2001
user.class : super-user
user.authentication.encrypted-password : $1$bB8UMoy/$Uk7/9QTMb6mRXINs4cFYD.
--------------------------------------------------
user.name : pwrage
user.uid : 2000
user.class : super-user
user.authentication.encrypted-password : $1$1YAYScOT$EVhU5CnxrSU5YuxAUtAXA.


In [5]:
interfaces: List[_Element] = config.xpath('//configuration/interfaces/interface')


for interface in interfaces:
    print('-'*50)  # print separator
    prefixes = []  # collect parent element tag names
    for element in interface.iter():  # iterates in document order
        if element.text is None:  # element has no text; children are other elements
            prefixes.append(element.tag)
        else:
            key = f"{'.'.join(prefixes)}.{element.tag}"
            value = element.text
            print(f"{key} : {value}")

--------------------------------------------------
interface.name : fe-0/0/0
interface.unit.name : 0
interface.unit.family.inet.dhcp-client.server-address : 192.168.1.1
--------------------------------------------------
interface.name : fe-0/0/1
interface.unit.name : 0
interface.unit.family.ethernet-switching.vlan.members : vlan-trust
--------------------------------------------------
interface.name : fe-0/0/2
interface.unit.name : 0
interface.unit.family.ethernet-switching.vlan.members : vlan-trust
--------------------------------------------------
interface.name : fe-0/0/3
interface.unit.name : 0
interface.unit.family.ethernet-switching.vlan.members : vlan-trust
--------------------------------------------------
interface.name : fe-0/0/4
interface.unit.name : 0
interface.unit.family.ethernet-switching.vlan.members : vlan-trust
--------------------------------------------------
interface.name : fe-0/0/5
interface.unit.name : 0
interface.unit.family.ethernet-switching.vlan.members : vl

In [6]:
with open(f"../data/{host}.xml", 'w') as fh:
    fh.write(config.data_xml)
